In [1]:
import sys
sys.path.append('./YOLOv8-Lightweight-Backbones')

# Import necessary functions, including:
* data preprocessing steps - to extract then example image for profiling
* profiling classes - classes that will use given model and collect statistics about performance in speed, accuracy, memory footprint, FPS and etc.

In [2]:
import numpy as np
from keras_cv import visualization
from data.prepate_data import run_data_preparation_pipeline
from models.csp_darknet import create_xs_csp_darknet_yolov8
from models.ghostnet_v2 import create_0_5_ghostnet_yolov8
from models.shufflenet_v2 import create_0_5_shufflenet_yolov8

from keras.callbacks import CSVLogger
from profiling.coco_metric_callback import EvaluateCOCOMetricsCallback
from profiling.profilers import ModelsProfiler, FlopsExtractor, InferenceProfiler, COCOMetricsCalculator
import warnings
warnings.filterwarnings("ignore")

num_classes = 2
input_shape = (640, 640, 3)


# Collection of random image

In [3]:
gathered_data = run_data_preparation_pipeline()
train_ds, val_ds = gathered_data['train_ds'], gathered_data['val_ds']
cat_mapping_r = gathered_data['cat_mapping_r']
cat_mapping = gathered_data['cat_mapping']
example_data_for_inf = val_ds.take(1)
example_data_for_inf = next(iter(example_data_for_inf))[0][:1]

Using Colab cache for faster access to the 'cars-video-object-tracking' dataset.
Path to dataset files: /kaggle/input/cars-video-object-tracking


# Loading model weights

In [4]:
model_shuffle = create_0_5_shufflenet_yolov8(num_classes=num_classes, input_shape=input_shape)
csp_darknet_yolo = create_xs_csp_darknet_yolov8(num_classes=num_classes)
ghost_model = create_0_5_ghostnet_yolov8(num_classes=num_classes, input_shape=input_shape)

csp_darknet_yolo.load_weights("./YOLOv8-Lightweight-Backbones/best_weights/best_csp_darknet_yolo.keras")
model_shuffle.load_weights("./YOLOv8-Lightweight-Backbones/best_weights/best_shuffle_yolo.keras")
ghost_model.load_weights("./YOLOv8-Lightweight-Backbones/best_weights/best_ghost_yolo.keras")

# Run profiling:
* inference profiler will collect FPS, speed and RAM footprint using GPU and CPU separately over 250 iterations
* FlopsExtractor will provide statistics about amount of model parameters and floating point operations
* COCOMetricsCalculator will extract accuracy results, including MaP over different thresholds and object sizes

In [6]:
profilers = [
    InferenceProfiler(repeats=250, device='GPU:0', warmup_steps=30),
    InferenceProfiler(repeats=250, device='CPU:0'),
    FlopsExtractor(),
    COCOMetricsCalculator(val_ds)
]
models = {
    'CSPDarkNet-YOLOv8': csp_darknet_yolo,
    'GhostNetv2-YOLOv8': ghost_model,
    'ShuffleNetV2-YOLOv8': model_shuffle,
}
general_profiles = ModelsProfiler(profilers=profilers)
profile_results = general_profiles.profile(models, example_data_for_inf)

RUNNING  <profiling.profilers.InferenceProfiler object at 0x7a481458da00> FOR MODEL CSPDarkNet-YOLOv8
RUNNING  <profiling.profilers.InferenceProfiler object at 0x7a4810691520> FOR MODEL CSPDarkNet-YOLOv8
RUNNING  <profiling.profilers.FlopsExtractor object at 0x7a48106919d0> FOR MODEL CSPDarkNet-YOLOv8


Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


RUNNING  <profiling.profilers.COCOMetricsCalculator object at 0x7a4810691730> FOR MODEL CSPDarkNet-YOLOv8
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
RUNNING  <profiling.profilers.InferenceProfiler object at 0x7a481458da00> FOR MODEL GhostNetv2-YOLOv8
RUNNING  <profiling.profilers.InferenceProfiler object at 0x7a4810691520> FOR MODEL GhostNetv2-YOLOv8
RUNNING  <profiling.profilers.FlopsExtractor object at 0x7a48106919d0> FOR MODEL GhostNetv2-YOLOv8
RUNNING  

In [8]:
profile_results.to_csv("profiling_results_cpu_and_gpu.csv")